In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import sqlite3

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [22]:

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/TV_Market_Analysis/data/raw/updated_tv_cleaned.csv')

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Load the DataFrame into the SQLite database
df.to_sql('tv_data', conn, index=False, if_exists='replace')

# Verify the table has been created by listing all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables


,name
0,tv_data


Run basic queries to understand the dataset

Next, we'll run some basic queries to get an understanding of the dataset. We'll start by fetching the first few rows of the table.

In [23]:
# Fetch the first 5 rows of the dataset
first_five_rows = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)
first_five_rows

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour
0,oneplus y1s 40 inch (101 cm) led full hd tv,oneplus,y1s 4,40,18990,Full HD,1920x1080,Black
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black


Get summary statistics of the dataset

To get a better understanding of the data, let's run a query to get summary statistics such as the count, average, minimum, and maximum values for the numeric columns.

In [24]:
# Query to get summary statistics
summary_statistics = pd.read_sql("""
SELECT
    COUNT(*) AS total_records,
    AVG(Inches) AS avg_inches,
    MIN(Inches) AS min_inches,
    MAX(Inches) AS max_inches,
    AVG(Price) AS avg_price,
    MIN(Price) AS min_price,
    MAX(Price) AS max_price
FROM tv_data;
""", conn)

summary_statistics

,total_records,avg_inches,min_inches,max_inches,avg_price,min_price,max_price
0,1061,51.411876,24,98,89344.210179,5190,1999990


Check for missing values

Next, let's check if there are any missing values in the dataset.

In [25]:
# Query to count missing values in each column
missing_values = pd.read_sql("""
SELECT
    SUM(CASE WHEN Name IS NULL THEN 1 ELSE 0 END) AS missing_name,
    SUM(CASE WHEN Brand IS NULL THEN 1 ELSE 0 END) AS missing_brand,
    SUM(CASE WHEN Model IS NULL THEN 1 ELSE 0 END) AS missing_model,
    SUM(CASE WHEN Inches IS NULL THEN 1 ELSE 0 END) AS missing_inches,
    SUM(CASE WHEN Price IS NULL THEN 1 ELSE 0 END) AS missing_price,
    SUM(CASE WHEN DisplayType IS NULL THEN 1 ELSE 0 END) AS missing_display_type,
    SUM(CASE WHEN Resolution IS NULL THEN 1 ELSE 0 END) AS missing_resolution,
    SUM(CASE WHEN Colour IS NULL THEN 1 ELSE 0 END) AS missing_colour
FROM tv_data;
""", conn)

missing_values

,missing_name,missing_brand,missing_model,missing_inches,missing_price,missing_display_type,missing_resolution,missing_colour
0,0,0,0,0,0,0,0,0


Identify and handle duplicates

Now, let's check if there are any duplicate records in the dataset.

In [26]:
# Query to count duplicate records
duplicate_records = pd.read_sql("""
SELECT
    Name,
    Brand,
    Model,
    Inches,
    Price,
    DisplayType,
    Resolution,
    Colour,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Name,
    Brand,
    Model,
    Inches,
    Price,
    DisplayType,
    Resolution,
    Colour
HAVING COUNT(*) > 1;
""", conn)

duplicate_records

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,count


**Summary**

We have successfully:

Loaded the dataset into an in-memory SQLite database.

Run basic queries to understand the dataset structure and contents.

Checked for missing values and found none.

Identified no duplicate records.

Analyzed the distribution of categorical variables.

In [27]:
# Query to get column names and data types
column_info = pd.read_sql("PRAGMA table_info(tv_data);", conn)

column_info

,cid,name,type,notnull,dflt_value,pk
0,0,Name,TEXT,0,None,0
1,1,Brand,TEXT,0,None,0
2,2,Model,TEXT,0,None,0
3,3,Inches,INTEGER,0,None,0
4,4,Price,INTEGER,0,None,0
5,5,DisplayType,TEXT,0,None,0
6,6,Resolution,TEXT,0,None,0
7,7,Colour,TEXT,0,None,0


In [28]:
# Query to get the first 5 rows of the Resolution column
first_five_resolutions = pd.read_sql("SELECT Resolution FROM tv_data LIMIT 5;", conn)

first_five_resolutions

,Resolution
0,1920x1080
1,1920x1080
2,1366x768
3,3840x2160
4,3840x2160


In [29]:
# Query to get the last 5 rows of the Resolution column
last_five_resolutions = pd.read_sql("SELECT Resolution FROM tv_data ORDER BY ROWID DESC LIMIT 5;", conn)

last_five_resolutions

,Resolution
0,1366x768
1,3840x2160
2,3840x2160
3,3840x2160
4,1336x768


In [30]:
# Query to get the value counts of the Resolution column
resolution_value_counts = pd.read_sql("""
SELECT
    Resolution,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Resolution
ORDER BY
    count DESC;
""", conn)

resolution_value_counts

,Resolution,count
0,3840x2160,731
1,1366x768,188
2,1920x1080,117
3,7680x4320,17
4,3840x2860,2
5,1336x768,2
6,8480x2160,1
7,3860x2160,1
8,384x2160,1
9,3480x2160,1


Update incorrect Resolution values

In [31]:
# Query to update incorrect Resolution values
conn.execute("""
UPDATE tv_data
SET Resolution = '1366x768'
WHERE Resolution = '1336x768';
""")

# Verify the update by checking the value counts again
resolution_value_counts_cleaned = pd.read_sql("""
SELECT
    Resolution,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Resolution
ORDER BY
    count DESC;
""", conn)

resolution_value_counts_cleaned

,Resolution,count
0,3840x2160,731
1,1366x768,190
2,1920x1080,117
3,7680x4320,17
4,3840x2860,2
5,8480x2160,1
6,3860x2160,1
7,384x2160,1
8,3480x2160,1


In [32]:
# Query to update incorrect Resolution values from "384x2160" to "3840x2160"
conn.execute("""
UPDATE tv_data
SET Resolution = '3840x2160'
WHERE Resolution = '384x2160';
""")

# Verify the update by checking the value counts again
resolution_value_counts_corrected = pd.read_sql("""
SELECT
    Resolution,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Resolution
ORDER BY
    count DESC;
""", conn)

resolution_value_counts_corrected

,Resolution,count
0,3840x2160,732
1,1366x768,190
2,1920x1080,117
3,7680x4320,17
4,3840x2860,2
5,8480x2160,1
6,3860x2160,1
7,3480x2160,1


In [33]:
# Add new columns for Width and Height
conn.execute("ALTER TABLE tv_data ADD COLUMN Width INTEGER;")
conn.execute("ALTER TABLE tv_data ADD COLUMN Height INTEGER;")

# Update the Width and Height columns based on the Resolution
conn.execute("""
UPDATE tv_data
SET Width = CAST(SUBSTR(Resolution, 1, INSTR(Resolution, 'x') - 1) AS INTEGER),
    Height = CAST(SUBSTR(Resolution, INSTR(Resolution, 'x') + 1) AS INTEGER);
""")

# Verify the update by fetching the first 5 rows
updated_tv_data = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,Width,Height
0,oneplus y1s 40 inch (101 cm) led full hd tv,oneplus,y1s 4,40,18990,Full HD,1920x1080,Black,1920,1080
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black,1920,1080
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black,1366,768
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black,3840,2160
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black,3840,2160


In [34]:
# Add new column for PPI
conn.execute("ALTER TABLE tv_data ADD COLUMN PPI REAL;")

# Update the PPI column based on the Width, Height, and Inches
conn.execute("""
UPDATE tv_data
SET PPI = ROUND(SQRT((Width * Width) + (Height * Height)) / Inches, 2);
""")

# Verify the update by fetching the first 5 rows
updated_tv_data_with_ppi = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data_with_ppi

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,Width,Height,PPI
0,oneplus y1s 40 inch (101 cm) led full hd tv,oneplus,y1s 4,40,18990,Full HD,1920x1080,Black,1920,1080,55.07
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black,1920,1080,68.84
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black,1366,768,48.97
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black,3840,2160,67.78
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black,3840,2160,102.46


In [35]:
# Query to convert all brand names to lowercase
conn.execute("UPDATE tv_data SET Brand = LOWER(Brand);")

# Verify the update by checking the value counts again
brand_value_counts_lowercase = pd.read_sql("""
SELECT
    Brand,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Brand
ORDER BY
    count DESC;
""", conn)

brand_value_counts_lowercase

,Brand,count
0,samsung,232
1,lg,151
2,tcl,73
3,sony,60
4,panasonic,33
5,xiaomi,30
6,motorola,30
7,haier,30
8,hisense,27
9,croma,27


In [36]:
# Save the updated dataset to a new CSV file
updated_csv_path = 'updated_tv_data_with_ppi.csv'
df_updated = pd.read_sql("SELECT * FROM tv_data;", conn)
df_updated.to_csv(updated_csv_path, index=False)

updated_csv_path


'updated_tv_data_with_ppi.csv'

In [37]:
# Add a new column 'Original_Brand' as a copy of the 'Brand' column
conn.execute("ALTER TABLE tv_data ADD COLUMN Original_Brand TEXT;")
conn.execute("UPDATE tv_data SET Original_Brand = Brand;")

# Verify the update by fetching the first 5 rows
updated_tv_data_with_original_brand = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data_with_original_brand

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,Width,Height,PPI,Original_Brand
0,oneplus y1s 40 inch (101 cm) led full hd tv,oneplus,y1s 4,40,18990,Full HD,1920x1080,Black,1920,1080,55.07,oneplus
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black,1920,1080,68.84,tcl
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black,1366,768,48.97,tcl
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black,3840,2160,67.78,sony
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black,3840,2160,102.46,sony


In [38]:
# Update the Brand column to title case
conn.execute("UPDATE tv_data SET Brand = UPPER(SUBSTR(Brand, 1, 1)) || LOWER(SUBSTR(Brand, 2));")

# Verify the update by fetching the first 5 rows
updated_tv_data_title_case = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data_title_case

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,Width,Height,PPI,Original_Brand
0,oneplus y1s 40 inch (101 cm) led full hd tv,Oneplus,y1s 4,40,18990,Full HD,1920x1080,Black,1920,1080,55.07,oneplus
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,Tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black,1920,1080,68.84,tcl
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,Tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black,1366,768,48.97,tcl
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,Sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black,3840,2160,67.78,sony
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,Sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black,3840,2160,102.46,sony


Create Price_Category column

We'll define the following bins for the Price column (in rupees):

Budget: 0 - 10,000

Affordable: 10,001 - 30,000

Mid-range: 30,001 - 60,000

High-end: 60,001 - 100,000

Premium: 100,001 - 200,000

Luxury: 200,001 and above

In [39]:
# Add new column for Price_Category
conn.execute("ALTER TABLE tv_data ADD COLUMN Price_Category TEXT;")

# Update the Price_Category column based on the Price
conn.execute("""
UPDATE tv_data
SET Price_Category = CASE
    WHEN Price <= 10000 THEN 'Budget'
    WHEN Price BETWEEN 10001 AND 30000 THEN 'Affordable'
    WHEN Price BETWEEN 30001 AND 60000 THEN 'Mid-range'
    WHEN Price BETWEEN 60001 AND 100000 THEN 'High-end'
    WHEN Price BETWEEN 100001 AND 200000 THEN 'Premium'
    ELSE 'Luxury'
END;
""")

# Verify the update by fetching the first 5 rows
updated_tv_data_with_price_category = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data_with_price_category

,Name,Brand,Model,Inches,Price,DisplayType,Resolution,Colour,Width,Height,PPI,Original_Brand,Price_Category
0,oneplus y1s 40 inch (101 cm) led full hd tv,Oneplus,y1s 4,40,18990,Full HD,1920x1080,Black,1920,1080,55.07,oneplus,Affordable
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,Tcl,32s5403af 3,32,13990,Full HD,1920x1080,Black,1920,1080,68.84,tcl,Affordable
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,Tcl,32s5400a 3,32,9990,HD-Ready,1366x768,Black,1366,768,48.97,tcl,Budget
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,Sony,bravia kd-65x74l 6,65,74990,4K,3840x2160,Black,3840,2160,67.78,sony,High-end
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,Sony,bravia kd-43x74k 4,43,39990,4K,3840x2160,Black,3840,2160,102.46,sony,Mid-range


In [40]:
# Step 1: Create a new table with the columns in the desired order
conn.execute("""
CREATE TABLE tv_data_reordered AS
SELECT
    Name,
    Brand,
    Model,
    Inches,
    Price,
    Price_Category,
    DisplayType,
    Resolution,
    Colour,
    Width,
    Height,
    PPI,
    Original_Brand
FROM tv_data;
""")

# Step 2: Drop the original table
conn.execute("DROP TABLE tv_data;")

# Step 3: Rename the new table to the original table name
conn.execute("ALTER TABLE tv_data_reordered RENAME TO tv_data;")

# Verify the update by fetching the first 5 rows
updated_tv_data_sql_reordered = pd.read_sql("SELECT * FROM tv_data LIMIT 5;", conn)

updated_tv_data_sql_reordered

,Name,Brand,Model,Inches,Price,Price_Category,DisplayType,Resolution,Colour,Width,Height,PPI,Original_Brand
0,oneplus y1s 40 inch (101 cm) led full hd tv,Oneplus,y1s 4,40,18990,Affordable,Full HD,1920x1080,Black,1920,1080,55.07,oneplus
1,tcl 32s5403af 32 inch (81 cm) led full hd tv,Tcl,32s5403af 3,32,13990,Affordable,Full HD,1920x1080,Black,1920,1080,68.84,tcl
2,tcl 32s5400a 32 inch (81 cm) led hd-ready tv,Tcl,32s5400a 3,32,9990,Budget,HD-Ready,1366x768,Black,1366,768,48.97,tcl
3,sony bravia kd-65x74l 65 inch (165 cm) led 4k tv,Sony,bravia kd-65x74l 6,65,74990,High-end,4K,3840x2160,Black,3840,2160,67.78,sony
4,sony bravia kd-43x74k 43 inch (109 cm) led 4k tv,Sony,bravia kd-43x74k 4,43,39990,Mid-range,4K,3840x2160,Black,3840,2160,102.46,sony


In [41]:
# Check data types of the dataset
data_types = pd.read_sql("PRAGMA table_info(tv_data);", conn)

data_types

,cid,name,type,notnull,dflt_value,pk
0,0,Name,TEXT,0,None,0
1,1,Brand,TEXT,0,None,0
2,2,Model,TEXT,0,None,0
3,3,Inches,INT,0,None,0
4,4,Price,INT,0,None,0
5,5,Price_Category,TEXT,0,None,0
6,6,DisplayType,TEXT,0,None,0
7,7,Resolution,TEXT,0,None,0
8,8,Colour,TEXT,0,None,0
9,9,Width,INT,0,None,0


In [42]:
# Query to get the value counts of the Price_Category column
price_category_value_counts = pd.read_sql("""
SELECT
    Price_Category,
    COUNT(*) AS count
FROM tv_data
GROUP BY
    Price_Category
ORDER BY
    count DESC;
""", conn)

price_category_value_counts

,Price_Category,count
0,Affordable,314
1,Mid-range,313
2,High-end,154
3,Premium,120
4,Luxury,93
5,Budget,67
